Things need to be fix
- `bitsandbytes` packages issue causing smaller model unable to be loaded
- Full model unquantized exceeds memory on Colab 

In [3]:
! pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 30.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
#  STEP 2: Setup Hugging Face Token
import os
# Replace "your_token_here" with your Hugging Face access token
os.environ["HUGGINGFACE_TOKEN"] = None
from huggingface_hub import login
login(token=os.environ["HUGGINGFACE_TOKEN"])

In [ ]:
#  STEP 3: Load Mistral 7B with 4-bit quantization
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "mistralai/Mistral-7B-Instruct-v0.1"
# The tokenizer and model should be loaded without the `token` parameter in `from_pretrained`
# if you have already logged in with your Hugging Face token using `login`.
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Move model to GPU
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

In [ ]:
business_case = """
Our retail company is expanding rapidly and we need better visibility into sales
performance across different regions.
We are currently using separate systems in each region and struggling to
generate consolidated reports.
Our main goal is to improve decision-making by integrating all sales data into one
centralized warehouse.
We also want to track KPIs like regional sales growth, top-performing products,
and customer retention rates.
This project is sponsored by the Sales Director and supported by the Regional
Managers.
"""

In [ ]:
#  STEP 5: Craft the prompt
prompt = f"""
You are a business analyst assistant. Given the following business case, extract:
1. Business Goals
2. Key Stakeholders
3. Important KPIs
Business Case:
{business_case}
"""

In [ ]:
#  STEP 6: Generate LLM response
inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(device)
outputs = model.generate(**inputs, max_new_tokens=300)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
#  STEP 7: Print response
print(" LLM Extracted Output:\n")
print(response)